In [1]:
#libraries
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

In [2]:
#variables
#date formatting is as follows YYYY-MM-DD"
 
start_date = '2018-07-05'
end_date = '2018-09-07'

#comparable and benchmarks 
comparable = 'AAPL'
benchmark1 = '^GSPC'
benchmark2 = '^IXIC'

In [3]:
#fetch data for comparable and benchmarks

comparable_data = yf.download(comparable, start=start_date, end=end_date, progress=False)
benchmark1_data = yf.download(benchmark1, start=start_date, end=end_date, progress=False)
benchmark2_data = yf.download(benchmark2, start=start_date, end=end_date, progress=False)

#reset index to use 'Date' as a column
comparable_data = comparable_data.reset_index()[['Date', 'Close']]
benchmark1_data = benchmark1_data.reset_index()[['Date', 'Close']]
benchmark2_data = benchmark2_data.reset_index()[['Date', 'Close']]

#rename columns for clarity
comparable_data.columns = ['Comparable #1 Dates', 'Comparable #1 Close Price']
benchmark1_data.columns = ['Benchmark #1 Dates', 'Benchmark #1 Price']
benchmark2_data.columns = ['Benchmark #2 Dates', 'Benchmark #2 Price']

#merge dates to align data
data = comparable_data.copy()

In [4]:
#comparable #1 Return

data['Comparable #1 Return'] = data['Comparable #1 Close Price'].apply(lambda x: np.nan if x == "" else x)
data['Comparable #1 Return'] = np.log(data['Comparable #1 Close Price'] / data['Comparable #1 Close Price'].shift(1))

# replace NaN with '-' for missing or first-row values
data['Comparable #1 Return'] = data['Comparable #1 Return'].fillna('-')

In [5]:
#returns for benchmark
benchmark1_data['Benchmark #1 Return'] = np.log(benchmark1_data['Benchmark #1 Price'] / benchmark1_data['Benchmark #1 Price'].shift(1))
benchmark2_data['Benchmark #2 Return'] = np.log(benchmark2_data['Benchmark #2 Price'] / benchmark2_data['Benchmark #2 Price'].shift(1))

# Merge benchmark returns into the main DataFrame
data = data.merge(benchmark1_data[['Benchmark #1 Dates', 'Benchmark #1 Return']], 
                  left_on='Comparable #1 Dates', right_on='Benchmark #1 Dates', how='left')
data = data.merge(benchmark2_data[['Benchmark #2 Dates', 'Benchmark #2 Return']], 
                  left_on='Comparable #1 Dates', right_on='Benchmark #2 Dates', how='left')

In [6]:
#comparable #1 Geodummy

data['Comparable #1 Geodummy'] = data['Comparable #1 Return'].apply(
    lambda x: '-' if x == '-' else x + 1
)

In [7]:
#comparable #1 Drawdown

data['Comparable #1 Drawdown'] = data['Comparable #1 Close Price'] - data['Comparable #1 Close Price'].cummax()

In [8]:
#comparable #1 Drawdown Recovery Length

recovery_length = 0
for i in range(len(data)):
    if data.loc[i, 'Comparable #1 Drawdown'] == 0:
        recovery_length = 0
    else:
        recovery_length += 1
    data.loc[i, 'Comparable #1 Drawdown Recovery Length'] = recovery_length

In [9]:
#comparable #1 Drawdown Recovery 2

data['Comparable #1 Drawdown Recovery'] = (
    data['Comparable #1 Drawdown'] / data['Comparable #1 Close Price'].cummax()
)

#replace NaN values in the first row or missing data with 0

data['Comparable #1 Drawdown Recovery'] = data['Comparable #1 Drawdown Recovery'].fillna(0)

In [10]:
#comparable #1 Date Check

data['Comparable #1 Date Check'] = data['Comparable #1 Dates'].apply(
    lambda x: "Pass" if (x in benchmark1_data['Benchmark #1 Dates'].values and x in benchmark2_data['Benchmark #2 Dates'].values) else "Fail"
)

In [ ]:
#difference of Comparable #1 and Benchmark #1 Returns

data['Difference of Comparable #1 and Benchmark #1 Returns'] = (
    data['Comparable #1 Return'] - data['Benchmark #1 Return']
)

#replace NaN values with "-" for clarity, if desired
data['Difference of Comparable #1 and Benchmark #1 Returns'] = data['Difference of Comparable #1 and Benchmark #1 Returns'].fillna("-")

In [12]:
#checking data

data.head(10)

,Comparable #1 Dates,Comparable #1 Close Price,Comparable #1 Return,Benchmark #1 Dates,Benchmark #1 Return,Benchmark #2 Dates,Benchmark #2 Return,Comparable #1 Geodummy,Comparable #1 Drawdown,Comparable #1 Drawdown Recovery Length,Comparable #1 Drawdown Recovery,Comparable #1 Date Check,Difference of Comparable #1 and Benchmark #1 Returns
0,2018-07-05,43.962238,-,2018-07-05,NaN,2018-07-05,NaN,-,0.000000,0.0,0.000000,Pass,-
1,2018-07-06,44.571651,0.013767,2018-07-06,0.008446,2018-07-06,0.013350,1.013767,0.000000,0.0,0.000000,Pass,0.005321
2,2018-07-09,45.190525,0.013789,2018-07-09,0.008784,2018-07-09,0.008781,1.013789,0.000000,0.0,0.000000,Pass,0.005005
3,2018-07-10,45.135994,-0.001207,2018-07-10,0.003467,2018-07-10,0.000387,0.998793,-0.054531,1.0,-0.001207,Pass,-0.004675
4,2018-07-11,44.550289,-0.013061,2018-07-11,-0.007119,2018-07-11,-0.005504,0.986939,-0.640236,2.0,-0.014167,Pass,-0.005942
5,2018-07-12,45.297230,0.016627,2018-07-12,0.008711,2018-07-12,0.013811,1.016627,0.000000,0.0,0.000000,Pass,0.007916
6,2018-07-13,45.368374,0.001569,2018-07-13,0.001079,2018-07-13,0.000263,1.001569,0.000000,0.0,0.000000,Pass,0.000491
7,2018-07-16,45.268780,-0.002198,2018-07-16,-0.001029,2018-07-16,-0.002592,0.997802,-0.099594,1.0,-0.002195,Pass,-0.001169
8,2018-07-17,45.396820,0.002824,2018-07-17,0.003966,2018-07-17,0.006309,1.002824,0.000000,0.0,0.000000,Pass,-0.001141
9,2018-07-18,45.147842,-0.0055,2018-07-18,0.002158,2018-07-18,-0.000087,0.9945,-0.248978,1.0,-0.005484,Pass,-0.007658
